当たり前だけど、embeddingが雑だと上手くいかない。

In [1]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

import gensim
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

# model = gensim.models.KeyedVectors.load_word2vec_format('./cc.en.300.vec.gz', binary=False)

INPUT_PATH = '../input'
OUTPUT_PATH = '../output'

In [12]:
data = pd.read_csv(os.path.join(INPUT_PATH, 'train.csv'))
data.head()

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e
1,E_000002eae2a589,Carioca Manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,BR,NaN,NaN,Brazilian Restaurants,P_d82910d8382a83
2,E_000007f24ebc95,ร้านตัดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,TH,NaN,NaN,Salons / Barbershops,P_b1066599e78477
3,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,NaN,Mobile Phone Shops,P_b2ed86905a4cd3
4,E_00001d92066153,Restaurante Casa Cofiño,43.338196,-4.326821,NaN,Caviedes,Cantabria,NaN,ES,NaN,NaN,Spanish Restaurants,P_809a884d4407fb


In [13]:
cats = data['categories'].fillna('').to_list()
cats = [x for cat in cats for x in cat.split(', ') if cat != '']

# Keep only letters
cats = [re.sub(r'[^a-zA-Z]', ' ', x) for x in cats]
        
# Delete multi space
cats = [re.sub('\\s+', ' ', x).strip() for x in cats]
cats = list(set(cats))

In [14]:
# embeddingは別で考えるので、今回は一旦、vocabにないものは省きます。
keys = list(model.key_to_index.keys())
categories = []
cnt = 0

for cat in cats:
    is_valid = True
    for word in cat.split(' '):
        if word not in keys:
            is_valid = False
            print(cat, ': ', word)
            cnt += 1
            break
    if is_valid:
        categories.append(cat)
        
assert len(categories) == len(cats) - cnt

NameError: name 'model' is not defined

# Embedding

In [46]:
# categories = ['Airports', 'Airport Trams', 'Airport Terminals', 'Airport Food Courts', 'Airport Lounges', 'Airport Services', 'Sorority Houses']

In [77]:
category_embeddings = []
for cat in categories:
    tmp = []
    for c in cat.split(' '):
        vec = model[c]
        tmp.append(vec)
    cat_vec = np.stack(tmp).sum(axis=0)
    cat_vec = torch.tensor(cat_vec)
    cat_vec = torch.nn.functional.normalize(cat_vec, p=2, dim=-1)
    
    category_embeddings.append(cat_vec)
category_embeddings = torch.stack(category_embeddings)


In [52]:
# category_embeddings = torch.tensor(category_embeddings)
# torch.cosine_similarity(category_embeddings[0], category_embeddings, dim=-1)

# Calc Distance

In [79]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CategoryEmbeddingDataset(Dataset):
    def __init__(self, category_embeddings):
        self.category_embeddings = category_embeddings

    def __len__(self):
        return len(self.category_embeddings)

    def __getitem__(self, idx):
        return self.category_embeddings[idx]
    
dataset = CategoryEmbeddingDataset(category_embeddings)
dataloader = DataLoader(dataset, batch_size=1000, shuffle=False)

In [80]:
similarities = []
for embeddings in dataloader:
    similarity = torch.cosine_similarity(embeddings.unsqueeze(1), torch.tensor(category_embeddings), dim=-1)
    similarities.append(similarity)
similarities = torch.concat(similarities)
similarities = similarities.numpy()

/var/folders/hk/k_9rvp7153g5n7xtm8clq6vw0000gp/T/ipykernel_22709/1596688489.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  similarity = torch.cosine_similarity(embeddings.unsqueeze(1), torch.tensor(category_embeddings), dim=-1)


In [3]:
# np.save(os.path.join(OUTPUT_PATH, 'similarities.npy'), similarities)

similarities = np.load(os.path.join(OUTPUT_PATH, 'similarities.npy'))

In [4]:
similarities = similarities * 10000000
similarities = 10000000 - similarities
similarities = similarities.astype(np.int64).tolist()

In [5]:
similarities[4]

[1789264,
 6518284,
 6645788,
 8444399,
 0,
 5729329,
 7618769,
 7239191,
 4575750,
 7440009,
 6696524,
 7545626,
 7705412,
 7162680,
 7422678,
 7231059,
 8892576,
 6860760,
 7689881,
 7139894,
 7102476,
 7344477,
 7200246,
 3683733,
 7390486,
 6236406,
 8303906,
 5521956,
 6945494,
 5251758,
 7557353,
 6274647,
 6858434,
 6154360,
 5540899,
 6076188,
 6049830,
 5694917,
 7707769,
 7799064,
 7470148,
 6390081,
 6694423,
 7581275,
 5347765,
 7321444,
 6189368,
 7005019,
 6778748,
 7255410,
 5070031,
 8272777,
 6144025,
 7673973,
 8115805,
 5761324,
 7989542,
 5372230,
 7667880,
 5326544,
 8015384,
 7672764,
 6836172,
 7709046,
 8230144,
 7762487,
 5044606,
 5077850,
 6849290,
 5535190,
 7646544,
 6943566,
 6805085,
 8107239,
 6738955,
 7934453,
 5114820,
 7336676,
 6956005,
 6334343,
 6901984,
 7360865,
 6191219,
 7235701,
 6926549,
 5763264,
 5541591,
 6416806,
 6131910,
 7524816,
 6522507,
 7141051,
 8639490,
 6765195,
 7398993,
 6842937,
 7800736,
 5128961,
 8391265,
 7579324,
 55032

In [68]:
categories[4]

'Travel Transport'

In [70]:
categories[-1]

'Newsstands'

# Calc TSP

In [6]:
manager = pywrapcp.RoutingIndexManager(len(similarities), 1, 0)
routing = pywrapcp.RoutingModel(manager)

In [7]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return similarities[from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [8]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [9]:
solution = routing.SolveWithParameters(search_parameters)

In [10]:
def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

routes = get_routes(solution, routing, manager)

In [15]:
tmp = []
for i, cat in zip(routes[0], categories):
    tmp.append([i, cat])

NameError: name 'categories' is not defined

In [42]:
df = pd.DataFrame(tmp, columns=['ix', 'category'])
df = df.sort_values('ix')

In [40]:
df[df['category']=='Airports']

,ix,category
341,525,Airports


In [43]:
df[(df['ix']>500) & (df['ix']<550)]

,ix,category
808,501,Thai Restaurants
93,502,Soccer Fields
529,503,College Libraries
33,504,Indonesian Meatball Places
586,505,Chocolate Shops
739,506,Modern European Restaurants
20,507,Israeli Restaurants
580,508,Boxing Gyms
233,509,Beijing Restaurants
39,510,Fried Chicken Joints


# BERT embeddings
https://colab.research.google.com/drive/1ZQvuAVwA3IjybezQOXnrXMGAnMyZRuPU#scrollTo=Zn0n2S-FWZih

In [18]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

100%|██████████| 231508/231508 [00:00<00:00, 517557.19B/s]


In [44]:
text = "Mobile Phone Shops"
marked_text = "[CLS] " + text + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [1] * len(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [45]:
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
token_vecs = encoded_layers[11][0]
sentence_embedding = torch.mean(token_vecs, dim=0)

In [48]:
# torch.cosine_similarity(a, b, dim=-1)